### This project is to understand how DTI works and I am using the datasets from kaggle to build a RF and XGBoost model for predicting if DTI will occur or not between a Drug and a Target protein.
#### My approach and thought process
#### For the Drug compunds, I will use the ReLeaSe dataset from kaggle.
#### Import and clean the data, convert SMILES to structural fingerprints using the RDKit library, I will consider  
#### - Extended-Connectivity Fingerprints (ECFP)
#### - Molecular ACCess System (MACCS) keys
#### - Topological Torsion Fingerprints
#### For the Protein sequences, I will use 
#### Convert the Protein sequences to OHE
#### Merge the extracted fingerprints 


### The dataset - De novo Drug Design Dataset - ReLeaSe Dataset

"pIC50" - is a measurement used in pharmacology and drug discovery to assess the potency of a compound in inhibiting a specific biological target or enzyme

"mol" - is a representation of a molecule in the RDKit library, which is a widely used cheminformatics software toolkit for working with molecular data. In this case, it indicates that the column "mol" contains RDKit Mol objects.

"num_atoms" - refers to the number of atoms in a molecule. In the context of cheminformatics and molecular data analysis, the "num_atoms" attribute represents the count of atoms present in a molecule. Atoms are the fundamental units that make up molecules. They can be elements such as carbon (C), hydrogen (H), oxygen (O), nitrogen (N), and many others. The number of atoms in a molecule is an important property that can provide insights into its size, complexity, and potential chemical interactions.

"logP" - is a measure of how a molecule interacts with different solvents. It indicates the molecule's preference for nonpolar (oily) or polar (water-like) environments. A higher logP value means the molecule is more likely to dissolve in oils and less likely to dissolve in water. This property is important in drug design as it affects a molecule's ability to be absorbed and distributed in the body

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/Users/samuelsetsofia/dev/DTI_Prediction/SMILES_Big_Data_Set.csv')
df.head()

,SMILES,pIC50,mol,num_atoms,logP
0,O=S(=O)(Nc1cccc(-c2cnc3ccccc3n2)c1)c1cccs1,4.26,<rdkit.Chem.rdchem.Mol object at 0x7f59df45bc30>,25,4.1591
1,O=c1cc(-c2nc(-c3ccc(-c4cn(CCP(=O)(O)O)nn4)cc3)...,4.34,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c9e0>,36,3.6743
2,NC(=O)c1ccc2c(c1)nc(C1CCC(O)CC1)n2CCCO,4.53,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cac0>,23,1.5361
3,NCCCn1c(C2CCNCC2)nc2cc(C(N)=O)ccc21,4.56,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cba0>,22,0.9510
4,CNC(=S)Nc1cccc(-c2cnc3ccccc3n2)c1,4.59,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c7b0>,21,3.2130


In [4]:
df.shape

(16087, 5)

In [5]:
df.describe

<bound method NDFrame.describe of                                                   SMILES  pIC50  \
0             O=S(=O)(Nc1cccc(-c2cnc3ccccc3n2)c1)c1cccs1   4.26   
1      O=c1cc(-c2nc(-c3ccc(-c4cn(CCP(=O)(O)O)nn4)cc3)...   4.34   
2                 NC(=O)c1ccc2c(c1)nc(C1CCC(O)CC1)n2CCCO   4.53   
3                    NCCCn1c(C2CCNCC2)nc2cc(C(N)=O)ccc21   4.56   
4                      CNC(=S)Nc1cccc(-c2cnc3ccccc3n2)c1   4.59   
...                                                  ...    ...   
16082               S=C(NN=C(c1ccccn1)c1ccccn1)Nc1ccccc1   0.00   
16083                            S=C=NCCCCCCCCCCc1ccccc1   0.00   
16084                              S=C=NCCCCCCCCc1ccccc1   0.00   
16085              S=c1[nH]nc(Cn2ccc3ccccc32)n1-c1ccccc1   0.00   
16086                   N=[N+]=NC=Cc1ccc([N+](=O)[O-])o1   0.02   

                                                    mol  num_atoms     logP  
0      <rdkit.Chem.rdchem.Mol object at 0x7f59df45bc30>         25  4.15910  
1    

In [6]:
df.describe()

,pIC50,num_atoms,logP
count,15037.000000,16087.000000,16087.000000
mean,0.998739,18.749984,2.256600
std,2.479588,8.428888,1.609861
min,0.000000,3.000000,-5.395600
25%,0.000000,13.000000,1.214900
50%,0.010000,17.000000,2.163200
75%,0.130000,23.000000,3.232440
max,10.970000,85.000000,15.879200


In [7]:
df.dtypes

SMILES        object
pIC50        float64
mol           object
num_atoms      int64
logP         float64
dtype: object

In [8]:
df.head()

,SMILES,pIC50,mol,num_atoms,logP
0,O=S(=O)(Nc1cccc(-c2cnc3ccccc3n2)c1)c1cccs1,4.26,<rdkit.Chem.rdchem.Mol object at 0x7f59df45bc30>,25,4.1591
1,O=c1cc(-c2nc(-c3ccc(-c4cn(CCP(=O)(O)O)nn4)cc3)...,4.34,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c9e0>,36,3.6743
2,NC(=O)c1ccc2c(c1)nc(C1CCC(O)CC1)n2CCCO,4.53,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cac0>,23,1.5361
3,NCCCn1c(C2CCNCC2)nc2cc(C(N)=O)ccc21,4.56,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cba0>,22,0.9510
4,CNC(=S)Nc1cccc(-c2cnc3ccccc3n2)c1,4.59,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c7b0>,21,3.2130
